In [3]:
from urllib.request import urlopen
from xml.etree import ElementTree as ET
import sqlite3
import re
import ssl
import time
import timeit

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [10]:
conn = sqlite3.connect('/mnt/d/Users/Taina/Documents/Python/Goodreads.sqlite')
cur = conn.cursor()

cur.executescript('''CREATE TABLE IF NOT EXISTS Books (
    id  INTEGER NOT NULL PRIMARY KEY UNIQUE,
    title TEXT,
    isbn INTEGER,
    publisher_id INTEGER,
    language_id INTEGER,
    rating INTEGER,
    rating_count INTEGER,
    review_count INTEGER,
    pages INTEGER,
    description TEXT
    );
    
    CREATE TABLE IF NOT EXISTS Authors (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS Book_Author (
    book_id INTEGER,
    author_id INTEGER,
    PRIMARY KEY(book_id, author_id)
    );
    
    CREATE TABLE IF NOT EXISTS Publishers (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    publisher TEXT UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS Language (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    language TEXT UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS Shelves (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name TEXT UNIQUE
    );
    
    CREATE TABLE IF NOT EXISTS Book_Shelf (
    book_id INTEGER,
    shelf_id INTEGER,
    count INTEGER,
    PRIMARY KEY(book_id, shelf_id));
    ''')

conn.commit()

In [5]:
def getInfo(book, item):
    try:
        return book.find(item).text
    except:
        return None
    
def getAuthors(book):
    #como um livro pode ter mais de um author criei um metodo separado
    try:
        #role poderia ser ilustrador ou tradutor, que eu não quero salvar
        return [t.find('./name').text for t in book.findall('./authors/author') if t.find('./role').text is None]
    except:
        return None
    
def getShelves(book):
    #como um livro pode ter mais de uma estante criei um metodo separado
    try:
        return [t.attrib for t in book.findall('./popular_shelves/shelf')]
    except:
        return None
    
def getBookInfo(book):

    title = getInfo(book, 'title')
    book_id = getInfo(book, 'id')
    isbn = getInfo(book, 'isbn13')
    authors = getAuthors(book)
    publisher = getInfo(book, 'publisher')
    language = getInfo(book, 'language_code')
    rating = getInfo(book, 'average_rating')
    rating_count = getInfo(book, 'ratings_count')
    review_count = getInfo(book, 'text_reviews_count')
    pages = getInfo(book, 'num_pages')
    description = getInfo(book, 'description')
    publication_year = getInfo(book, 'publication_year')
    shelves = getShelves(book)
    
    return title, book_id, isbn, authors, publisher, language, rating, rating_count, review_count, pages, description, publication_year, shelves

In [6]:
def insertBook(book_id):
        
    try:
        xml = urlopen('https://www.goodreads.com/book/show/' + str(book_id) + '.xml?key=OkeUuzu8EVFAaFJYIv6sg', context = ctx)
        book = ET.fromstring(xml.read()).find('book')
    except:
        print('book', str(book_id), 'not returned...')
        return
    
    title, book_id, isbn, authors, publisher, language, rating, rating_count, review_count, pages, description, publication_year, shelves = getBookInfo(book)

    if not publisher is None:
        cur.execute('''INSERT OR IGNORE INTO Publishers (publisher) VALUES ( ? )''', (publisher, ))
        cur.execute('SELECT id FROM Publishers WHERE publisher = ? ', (publisher, ))
        publisher_id = cur.fetchone()[0]
    else:
        publisher_id = None
        
    if not language is None:
        cur.execute('''INSERT OR IGNORE INTO Language (language) VALUES ( ? )''', (language, ))  
        cur.execute('SELECT id FROM Language WHERE language = ? ', (language, ))
        language_id = cur.fetchone()[0]
    else:
        language_id = None

    cur.execute('''INSERT OR IGNORE INTO Books (id, title, isbn, publisher_id, language_id, 
                    rating, rating_count, review_count, pages, description)
                    VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ? )''',
                (book_id, title, isbn, publisher_id, language_id,
                rating, rating_count, review_count, pages, description))

    for author in authors:
        cur.execute('''INSERT OR IGNORE INTO Authors (name) VALUES ( ? )''', (author, ))
        cur.execute('SELECT id FROM Authors WHERE name = ? ', (author, ))
        author_id = cur.fetchone()[0]    
        cur.execute('''INSERT OR IGNORE INTO Book_Author (book_id, author_id) VALUES ( ?, ? )''', (book_id, author_id))

    for shelf in shelves:
        cur.execute('''INSERT OR IGNORE INTO Shelves (name) VALUES ( ? )''', (shelf['name'], ))
        cur.execute('SELECT id FROM Shelves WHERE name = ? ', (shelf['name'], ))
        shelf_id = cur.fetchone()[0]    
        cur.execute('''INSERT OR IGNORE INTO Book_Shelf (book_id, shelf_id, count) VALUES ( ?, ?, ? )''', (book_id, shelf_id, shelf['count']))

    print('book', str(book_id), 'added!')
        
    conn.commit()
    time.sleep(1)

In [11]:
cur.execute('SELECT link FROM Links WHERE link LIKE "/book/show/%"')

for link, in cur.fetchall():
    book_id = re.findall(r'/book/show/([0-9]+)', link)[0]
    
    #olhando se o livro já está na base    
    cur.execute('SELECT id FROM Books WHERE id  = ? ', (book_id, ))
    retrieved = cur.fetchone()
    
    if not(retrieved is None):
        print('book', retrieved[0], 'is already on the database')        
        continue
    
    insertBook(book_id)
    

book 1 is already on the database
book 10000270 is already on the database
book 1000548 is already on the database
book 1001282 is already on the database
book 10016013 is already on the database
book 100247 is already on the database
book 10025007 is already on the database
book 10025007 is already on the database
book 10025656 is already on the database
book 100285 is already on the database
book 10031701 is already on the database
book 100322 is already on the database
book 10032870 is already on the database
book 100365 is already on the database
book 10037 is already on the database
book 10048459 is already on the database
book 10048874 is already on the database
book 10048874 is already on the database
book 10066 is already on the database
book 10069432 is already on the database
book 10073506 is already on the database
book 1007504 is already on the database
book 10080974 is already on the database
book 10082454 is already on the database
book 10083474 is already on the database

book 1142691 is already on the database
book 114316 is already on the database
book 114345 is already on the database
book 114437 is already on the database
book 11447921 is already on the database
book 11448281 is already on the database
book 11455096 is already on the database
book 11455096 is already on the database
book 11459269 is already on the database
book 11468074 is already on the database
book 114683 is already on the database
book 11468377 is already on the database
book 11468377 is already on the database
book 114707 is already on the database
book 11473255 is already on the database
book 11475038 is already on the database
book 11475039 is already on the database
book 11476593 is already on the database
book 11477648 is already on the database
book 11484681 is already on the database
book 11486 is already on the database
book 1148695 is already on the database
book 1148721 is already on the database
book 11488123 is already on the database
book 114916 is already on the da

book 12990480 is already on the database
book 12993856 is already on the database
book 12995185 is already on the database
book 12996 is already on the database
book 129972 is already on the database
book 129984 is already on the database
book 13 is already on the database
book 1301 is already on the database
book 130115 is already on the database
book 13018514 is already on the database
book 130200 is already on the database
book 130206 is already on the database
book 13021874 is already on the database
book 1302575 is already on the database
book 130312 is already on the database
book 130346 is already on the database
book 13034968 is already on the database
book 13035050 is already on the database
book 13035050 is already on the database
book 13038680 is already on the database
book 13040826 is already on the database
book 13042002 is already on the database
book 13043254 is already on the database
book 130440 is already on the database
book 13044805 is already on the database
book 

book  13501210  not returned...
book 13501633 is already on the database
book 13503109 is already on the database
book 13503544 is already on the database
book 13504472 is already on the database
book 13506310 is already on the database
book 13506703 is already on the database
book 13508415 is already on the database
book 13508607 is already on the database
book 13508897 is already on the database
book 13509563 is already on the database
book 13510444 is already on the database
book 13511532 is already on the database
book 1351174 is already on the database
book 13513038 is already on the database
book 13513868 is already on the database
book 13516846 is already on the database
book 13516846 is already on the database
book 1351707 is already on the database
book 13517212 is already on the database
book 13526165 is already on the database
book 13527860 is already on the database
book 13528319 is already on the database
book 13528381 is already on the database
book 13528381 is already on

book 1511910 is already on the database
book 1512568 is already on the database
book 151374 is already on the database
book 1514230 is already on the database
book 1514619 is already on the database
book 151488 is already on the database
book 1515231 is already on the database
book 1516 is already on the database
book 1517929 is already on the database
book 1519 is already on the database
book 151926 is already on the database
book 151928 is already on the database
book 15195 is already on the database
book 15196 is already on the database
book 1519670 is already on the database
book 152 is already on the database
book 152004 is already on the database
book 1522004 is already on the database
book 1523559 is already on the database
book 152380 is already on the database
book 15241 is already on the database
book 15241 is already on the database
book 152519 is already on the database
book 152660 is already on the database
book 15269524 is already on the database
book 15275304 is already 

...

book  67599  added!
book  6763730  added!
book 6763730 is already on the database
book  6763961  added!
book  6764211  added!
book  676640  added!
book  6769151  added!
book  676920  added!
book  6773012  added!
book  6775249  added!
book  6782468  added!
book  6783157  added!
book  6785671  added!
book  67864  added!
book  67891  added!
book  67896  added!
book  6792458  added!
book  6794  added!
book  67955  added!
book  67959  added!
book  67966  added!
book  6797849  added!
book  6798263  added!
book  6798553  added!
book  6800761  added!
book  68021  added!
book  680321  added!
book  6804268  added!
book  68092  added!
book  68099  added!
book  68143  added!
book  6819  added!
book  681941  added!
book  68210  added!
book  682128  added!
book  682793  added!
book  682804  added!
book  682920  added!
book  683096  added!
book  6832350  added!
book  6837103  added!
book 6837103 is already on the database
book  6839093  added!
book  68427  added!
book 68427 is already on the database

book  7494  added!
book  7494704  added!
book  7495987  added!
book  749620  added!
book  749622  added!
book  74998  added!
book  7501962  added!
book  750407  added!
book  7504179  added!
book  7507944  added!
book  7508561  added!
book  7514925  added!
book  7519231  added!
book  7519472  added!
book  7519640  added!
book  752123  added!
book  752943  added!
book  7531  added!
book  7531207  added!
book  753281  added!
book  7539751  added!
book  754100  added!
book  7542399  added!
book  7542823  added!
book  75508  added!
book  7557045  added!
book  7557392  added!
book  7568017  added!
book  7568425  added!
book  758643  added!
book  7588  added!
book  7597  added!
book  7597850  added!
book  7599029  added!
book  7600922  added!
book  7600924  added!
book 7600924 is already on the database
book  7603  added!
book  7604  added!
book  7607300  added!
book  7607392  added!
book  7608  added!
book  7608791  added!
book  76120  added!
book  7613  added!
book  7614793  added!
book  76

book  808550  added!
book  8086  added!
book  8086216  added!
book  8086283  added!
book  8089  added!
book  80890  added!
book  8091  added!
book  8098538  added!
book  8101452  added!
book  8103876  added!
book  81069  added!
book  810881  added!
book  8109709  added!
book  8114678  added!
book  81153  added!
book  8116091  added!
book  8119809  added!
book  8120173  added!
book  8122546  added!
book  81227  added!
book  8123901  added!
book  812404  added!
book  8127  added!
book 8127 is already on the database
book  8130027  added!
book  8130077  added!
book  8130467  added!
book  8131469  added!
book  813214  added!
book  8133097  added!
book  8135928  added!
book  8136599  added!
book  8139007  added!
book  8139820  added!
book  8140650  added!
book  8140731  added!
book  8143909  added!
book  8145882  added!
book  8146541  added!
book  8147  added!
book  8148686  added!
book  8149125  added!
book  8151  added!
book  8152537  added!
book  815309  added!
book  8154  added!
book  8

book  879637  added!
book  87985  added!
book  879989  added!
book  879990  added!
book  88061  added!
book  88071  added!
book  88077  added!
book  8812221  added!
book  8814993  added!
book 8814993 is already on the database
book  881528  added!
book  881710  added!
book  88247  added!
book  882988  added!
book  883217  added!
book  883235  added!
book 883235 is already on the database
book  8840017  added!
book  88442  added!
book  8844645  added!
book  884604  added!
book  884701  added!
book  8848689  added!
book  8849382  added!
book 8849382 is already on the database
book  8850140  added!
book  8852  added!
book  885494  added!
book  8855321  added!
book  885541  added!
book  8857310  added!
book  8864548  added!
book  8874743  added!
book  887797  added!
book  8882027  added!
book  8883358  added!
book 8883358 is already on the database
book  8884616  added!
book  8889080  added!
book  8893424  added!
book  889744  added!
book  890  added!
book  890210  added!
book  8908  added

book  9639970  added!
book  9640024  added!
book  9640626  added!
book  9641637  added!
book  9642370  not returned...
book  9643469  added!
book  9644151  added!
book  9646  added!
book  9647532  added!
book 9647532 is already on the database
book  9648068  added!
book 9648068 is already on the database
book 9648068 is already on the database
book 9648068 is already on the database
book 9648068 is already on the database
book  9648229  added!
book  964899  added!
book  9650  added!
book  96562  added!
book  9656600  added!
book  965960  added!
book  9659607  added!
book  9662481  added!
book  9663397  added!
book  9666870  added!
book  9672990  added!
book  9673436  added!
book  967629  added!
book  9677397  added!
book  968  added!
book 968 is already on the database
book  9680114  added!
book 9680114 is already on the database
book  9680403  added!
book  9680718  added!
book 9680718 is already on the database
book  968403  added!
book  9684523  added!
book  9685046  added!
book  968

In [12]:
conn.close()